In [1]:
# ==========================================================
# Artwork Analyzer v19 — Dynamic ROI-Only Pipeline (Stable)
# ==========================================================
# • Fully local (OpenCV + NumPy + Pillow)
# • Detects 2–5 adaptive Regions of Interest (ROIs)
# • Each ROI overlay includes its own bespoke paragraph
# • Combined ROI overlay includes all summaries
# • Outputs: ROI images + combined overlay + summary.txt + metrics.json
# ==========================================================

import os, json, time, math, textwrap
import numpy as np
import cv2
from PIL import Image
from math import atan2, degrees
from pathlib import Path

# ----------------------------------------------------------
# Utility Functions
# ----------------------------------------------------------

def ensure_dir(p):
    """Create directory (and parents) reliably."""
    Path(p).mkdir(parents=True, exist_ok=True)

def load_rgb(path):
    """Load image as RGB numpy array."""
    img = Image.open(path).convert("RGB")
    return np.array(img)

def to_uint8(img_f32):
    """Convert float [0, 1] → uint8 safely."""
    return np.clip(img_f32 * 255, 0, 255).astype(np.uint8)

def safe_gamma(img_u8):
    """Soft gamma correction for display readability."""
    img = img_u8.astype(np.float32) / 255.0
    mean_bright = float(img.mean())
    if mean_bright < 0.25:
        img = np.power(img, 0.75)
    elif mean_bright > 0.8:
        img = np.power(img, 1.1)
    return to_uint8(np.clip(img, 0, 1))

def resize_for_fidelity(img, target_pixels=1_600_000):
    """Upscale small artworks to improve analysis fidelity."""
    H, W = img.shape[:2]
    cur = H * W
    if cur >= target_pixels:
        return img, 1.0
    scale = math.sqrt(target_pixels / cur)
    newW, newH = int(W * scale), int(H * scale)
    resized = cv2.resize(img, (newW, newH), interpolation=cv2.INTER_CUBIC)
    return resized, scale

def json_safe(obj):
    """Convert numpy objects into JSON-serializable types."""
    if isinstance(obj, dict):
        return {k: json_safe(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [json_safe(x) for x in obj]
    elif isinstance(obj, (np.generic, np.number)):
        return obj.item()
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj
# ==========================================================
# Color & Texture Analysis (local only)
# ==========================================================

def ciede2000(Lab1, Lab2):
    """CIEDE2000 color difference between two LAB triplets."""
    L1,a1,b1 = Lab1; L2,a2,b2 = Lab2
    C1 = np.sqrt(a1*a1 + b1*b1); C2 = np.sqrt(a2*a2 + b2*b2)
    avg_C = (C1 + C2) / 2.0
    G = 0.5 * (1 - np.sqrt((avg_C**7) / ((avg_C**7) + 25**7)))
    a1p, a2p = (1 + G)*a1, (1 + G)*a2
    C1p, C2p = np.sqrt(a1p*a1p + b1*b1), np.sqrt(a2p*a2p + b2*b2)
    h1p = (np.degrees(np.arctan2(b1,a1p)) + 360) % 360
    h2p = (np.degrees(np.arctan2(b2,a2p)) + 360) % 360
    dLp, dCp = L2 - L1, C2p - C1p
    dhp = h2p - h1p
    if C1p*C2p == 0: dhp = 0
    elif dhp > 180: dhp -= 360
    elif dhp < -180: dhp += 360
    dHp = 2*np.sqrt(C1p*C2p) * np.sin(np.radians(dhp)/2)
    avg_Lp = (L1 + L2)/2.0
    avg_Cp = (C1p + C2p)/2.0
    avg_hp = (h1p + h2p)/2.0 if abs(h1p - h2p) <= 180 else (h1p + h2p + 360)/2.0
    T = (1 - 0.17*np.cos(np.radians(avg_hp - 30))
         + 0.24*np.cos(np.radians(2*avg_hp))
         + 0.32*np.cos(np.radians(3*avg_hp + 6))
         - 0.20*np.cos(np.radians(4*avg_hp - 63)))
    d_ro = 30*np.exp(-((avg_hp - 275)/25)**2)
    Rc = 2*np.sqrt((avg_Cp**7)/((avg_Cp**7) + 25**7))
    Sl = 1 + (0.015*(avg_Lp - 50)**2)/np.sqrt(20 + (avg_Lp - 50)**2)
    Sc, Sh = 1 + 0.045*avg_Cp, 1 + 0.015*avg_Cp*T
    Rt = -np.sin(np.radians(2*d_ro))*Rc
    return float(np.sqrt((dLp/Sl)**2 + (dCp/Sc)**2 + (dHp/Sh)**2 + Rt*(dCp/Sc)*(dHp/Sh)))

def color_analysis(img, k=8):
    """Extract main palette, color diversity, and harmony."""
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    H, W = lab.shape[:2]
    Z = lab.reshape(-1,3).astype(np.float32)
    crit = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 60, 0.2)
    _, labels, centers = cv2.kmeans(Z, k, None, crit, 6, cv2.KMEANS_PP_CENTERS)
    centers = centers.astype(np.float32)
    counts = np.bincount(labels.flatten(), minlength=k).astype(np.float32)
    frac = counts / (H*W + 1e-6)

    a,b = centers[:,1]-128, centers[:,2]-128
    hues = (np.degrees(np.arctan2(b,a)) + 360) % 360
    warm_ratio = float(((hues<90)|(hues>330)).sum()/len(hues))
    cool_ratio = float(((hues>180)&(hues<300)).sum()/len(hues))
    if np.ptp(hues) < 30:
        harmony = "analogous"
    elif any(abs(hues[i]-hues[j])%360>150 for i in range(len(hues)) for j in range(i+1,len(hues))):
        harmony = "complementary"
    else:
        harmony = "mixed"

    dEs = [ciede2000(centers[i], centers[j]) for i in range(len(centers)) for j in range(i+1,len(centers))]
    mean_dE = float(np.mean(dEs))
    entropy = -float(np.sum(frac[frac>0]*np.log2(frac[frac>0])))

    return dict(
        palette_lab=centers.tolist(),
        mean_deltaE=mean_dE,
        warm_ratio=warm_ratio,
        cool_ratio=cool_ratio,
        harmony=harmony,
        entropy=entropy
    )

def texture_analysis(gray):
    """Basic surface texture measure (entropy + anisotropy)."""
    gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1)
    mag = np.sqrt(gx*gx + gy*gy)
    ang = (np.degrees(np.arctan2(gy, gx)) + 180) % 180
    hist, _ = np.histogram(ang, bins=18, range=(0,180))
    anisotropy = float((hist.max() - hist.min()) / (hist.mean() + 1e-6))
    mag_norm = (mag - mag.min()) / (mag.max() - mag.min() + 1e-6)
    return dict(
        texture_map=mag_norm,
        texture_anisotropy=anisotropy,
        texture_mean=float(mag_norm.mean()),
        texture_std=float(mag_norm.std())
    )
# ==========================================================
# Saliency & Composition Analysis
# ==========================================================

def spectral_saliency(gray):
    """Compute spectral residual saliency map (frequency domain)."""
    grayf = gray.astype(np.float32)
    fft = np.fft.fft2(grayf)
    log_amp = np.log(np.abs(fft) + 1e-8)
    phase = np.angle(fft)
    spectral_residual = log_amp - cv2.blur(log_amp, (3,3))
    saliency = np.abs(np.fft.ifft2(np.exp(spectral_residual + 1j*phase)))**2
    saliency = cv2.GaussianBlur(saliency, (7,7), 2.0)
    saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-6)
    return saliency

def saliency_analysis(img):
    """Find areas that naturally draw attention."""
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sal = spectral_saliency(gray)
    m = cv2.threshold((sal*255).astype(np.uint8), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    cnts,_ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if cnts:
        x,y,w,h = cv2.boundingRect(max(cnts, key=cv2.contourArea))
    else:
        H,W = gray.shape
        x,y,w,h = W//4,H//4,W//2,H//2
    Y,X = np.indices(sal.shape)
    wgt = sal + 1e-6
    cx, cy = int((X*wgt).sum()/wgt.sum()), int((Y*wgt).sum()/wgt.sum())
    return dict(
        saliency_map=sal,
        focal=(cx,cy),
        subject_bbox=(x,y,w,h),
        subject_mask=m
    )

def composition_analysis(img, focal):
    """Detect structure using edges and Hough lines."""
    g = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(g, 80, 180)
    linesP = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=60, maxLineGap=12)
    lines = []
    if linesP is not None:
        for l in linesP[:200]:
            x1,y1,x2,y2 = map(int, l[0])
            lines.append((x1,y1,x2,y2))

    H, W = g.shape
    mid = W // 2
    left = g[:, :mid].astype(np.float32)
    right = np.fliplr(g[:, mid:].astype(np.float32))
    min_w = min(left.shape[1], right.shape[1])
    if min_w > 0:
        left = left[:, :min_w]
        right = right[:, :min_w]
        mse = float(np.mean((left - right) ** 2))
    else:
        mse = 0.0

    symmetry = 1.0 - min(1.0, mse * 3.0)
    lw = float(np.sum(edges[:, :mid]))
    rw = float(np.sum(edges[:, mid:]))
    balance = (rw - lw) / (rw + lw + 1e-6)

    fx, fy = focal
    thirds_pts = [(W/3, H/3), (2*W/3, H/3), (W/3, 2*H/3), (2*W/3, 2*H/3)]
    diag = np.hypot(W, H)
    thirds_delta = min(np.hypot(fx - x, fy - y) for x, y in thirds_pts) / (diag + 1e-6)

    return dict(
        lines=lines,
        symmetry=symmetry,
        balance=balance,
        thirds_delta=thirds_delta
    )
# ==========================================================
# Illumination & Light Direction Analysis
# ==========================================================

def illumination_analysis(img):
    """Analyze light direction and tonal skew in LAB space."""
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    L = lab[:,:,0].astype(np.float32)
    mu, sigma = L.mean(), L.std() + 1e-6
    skew = float((((L - mu) / sigma) ** 3).mean())

    gx = cv2.Sobel(L, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(L, cv2.CV_32F, 0, 1)
    hi_mask = (L > np.percentile(L, 90)).astype(np.uint8)
    if hi_mask.sum() > 0:
        gx_sum = float((gx * hi_mask).sum())
        gy_sum = float((gy * hi_mask).sum())
        ang = float((degrees(atan2(-gy_sum, -gx_sum)) + 360) % 360)
    else:
        ang = None

    return dict(
        light_skew=skew,
        light_direction_deg=ang
    )

def light_description(I):
    """Create friendly narrative description of illumination."""
    skew = I.get("light_skew", 0)
    ang = I.get("light_direction_deg", None)

    if ang is None:
        direction_phrase = "diffused evenly across the surface"
    elif 45 <= ang <= 135:
        direction_phrase = "entering softly from the top"
    elif 135 < ang <= 225:
        direction_phrase = "settling in from the left"
    elif 225 < ang <= 315:
        direction_phrase = "flowing across from the bottom"
    else:
        direction_phrase = "illuminating gently from the right"

    if skew > 0.6:
        tone_phrase = "bright and airy"
    elif skew < -0.6:
        tone_phrase = "dim and contemplative"
    else:
        tone_phrase = "balanced in tone"

    return f"The overall lighting feels {tone_phrase}, {direction_phrase}."
# ==========================================================
# ROI Interest Map Construction (Dynamic Fusion)
# ==========================================================

def _norm01(x):
    x = np.asarray(x, np.float32)
    mn, mx = x.min(), x.max()
    if mx - mn < 1e-6:
        return np.zeros_like(x, np.float32)
    return (x - mn) / (mx - mn)

def _gaussian_bump(H, W, cx, cy, sigma_frac=0.25):
    """Return a soft Gaussian map centered at (cx,cy)."""
    yy, xx = np.indices((H, W))
    d2 = (xx - cx)**2 + (yy - cy)**2
    diag = np.hypot(H, W)
    sigma = sigma_frac * diag
    g = np.exp(-d2 / (2.0 * sigma**2 + 1e-6))
    return _norm01(g)
# ==========================================================
# Text Wrapping Helper
# ==========================================================
import textwrap

def wrap_text_lines(text, width_chars=48):
    """
    Cleanly wrap text to multiple lines for OpenCV rendering.
    Keeps summaries readable and avoids overflow.
    """
    text = text.strip().replace("\n", " ")
    lines = textwrap.wrap(text, width=width_chars)
    return [ln.strip() for ln in lines if ln.strip()]


def _line_density_map(H, W, lines, blur=2.0):
    """Rasterize all composition lines into a soft density map."""
    m = np.zeros((H, W), np.uint8)
    if lines:
        for (x1,y1,x2,y2) in lines:
            cv2.line(m, (x1,y1), (x2,y2), 255, 1, cv2.LINE_AA)
    m = cv2.GaussianBlur(m, (0,0), blur)
    return _norm01(m)

def roi_interest_map(C, T, S, P, I, shape):
    """
    Combine all analytic maps into a unified interest heatmap.
    Dynamically weights signals depending on image complexity.
    """
    H, W = shape
    sal = _norm01(S.get("saliency_map", np.zeros((H, W), np.float32)))
    tex = _norm01(T.get("texture_map", np.zeros((H, W), np.float32)))
    line_density = _line_density_map(H, W, P.get("lines", []))
    contrast_proxy = np.zeros_like(sal)
    focal = S.get("focal", (W//2, H//2))
    prox_focal = _gaussian_bump(H, W, *focal, sigma_frac=0.25)
    prox_light = _gaussian_bump(H, W, W/2, H/2, sigma_frac=0.3) if I.get("light_direction_deg") else np.zeros_like(sal)

    # Adaptive weighting with robust defaults
    tex_std = float(T.get("texture_std", 0.1))
    mean_dE = float(C.get("mean_deltaE", 10.0))
    line_complex = np.tanh(len(P.get("lines", [])) / 150.0)
    tex_complex = np.tanh(tex_std * 4.0)
    contrast_wt = np.tanh(mean_dE / 25.0)

    w_sal = 0.4
    w_tex = 0.25 * (1 + 0.2*tex_complex)
    w_line = 0.15 * (1 + 0.2*line_complex)
    w_contrast = 0.15 * (1 + 0.2*contrast_wt)
    w_light = 0.05

    w_sum = w_sal + w_tex + w_line + w_contrast + w_light
    w_sal /= w_sum; w_tex /= w_sum; w_line /= w_sum; w_contrast /= w_sum; w_light /= w_sum

    fused = (w_sal * sal +
             w_tex * tex +
             w_line * line_density +
             w_contrast * contrast_proxy +
             w_light * prox_light +
             0.1 * prox_focal)

    fused = cv2.GaussianBlur(fused, (0,0), 1.0)
    return _norm01(fused)
# ==========================================================
# ROI Detection (Peaks + Separation) & Per-Region Factors
# ==========================================================

def _prepare_roi_inputs(C, T, S, P, I, shape):
    """Normalize and adapt analysis results for ROI fusion."""
    H, W = shape

    # Texture map
    tmap = T.get("texture_map", np.zeros((H, W), np.float32))
    if tmap.shape[:2] != (H, W):
        tmap = cv2.resize(tmap, (W, H), interpolation=cv2.INTER_CUBIC)
    T2 = dict(texture_map=_norm01(tmap), texture_std=T.get("texture_std", 0.1))

    # Contrast proxy
    cmap = C.get("contrast_map", np.zeros((H, W), np.float32))
    if cmap.shape[:2] != (H, W):
        cmap = cv2.resize(cmap, (W, H), interpolation=cv2.INTER_CUBIC)
    C2 = dict(mean_deltaE=float(C.get("mean_deltaE", 10.0)),
              contrast_map=_norm01(cmap))

    # Saliency
    smap = S.get("saliency_map", np.zeros((H, W), np.float32))
    if smap.shape[:2] != (H, W):
        smap = cv2.resize(smap, (W, H), interpolation=cv2.INTER_CUBIC)
    S2 = dict(saliency_map=_norm01(smap),
              focal=S.get("focal", (W//2, H//2)))

    # Composition
    P2 = dict(lines=P.get("lines", []),
              vanishing_point=P.get("vanishing_point", (W/2, H/2)))

    # Illumination
    I2 = dict(light_direction_deg=I.get("light_direction_deg", None))

    return C2, T2, S2, P2, I2


def _roi_component_maps(C2, T2, S2, P2, I2, shape):
    """Recreate component maps used in interest fusion."""
    H, W = shape
    sal = _norm01(S2["saliency_map"])
    tex = _norm01(T2["texture_map"])
    con = _norm01(C2["contrast_map"])

    fx, fy = S2["focal"]
    prox_focal = _gaussian_bump(H, W, fx, fy, sigma_frac=0.23)
    vx, vy = P2.get("vanishing_point", (W/2, H/2))
    prox_vp = _gaussian_bump(H, W, vx, vy, sigma_frac=0.28)
    line_density = _line_density_map(H, W, P2.get("lines", []))

    return dict(sal=sal, tex=tex, con=con,
                prox_focal=prox_focal, prox_vp=prox_vp, line=line_density)


def _find_peak_centers(interest, min_dist):
    """Find peak centers in the interest map with distance constraint."""
    H, W = interest.shape
    blur = cv2.GaussianBlur(interest, (0, 0), 1.2)
    maxf = cv2.dilate(blur, np.ones((5,5), np.uint8))
    peaks = (blur >= maxf - 1e-6).astype(np.uint8)
    thr = np.percentile(blur, 92)
    peaks = (peaks & (blur >= thr)).astype(np.uint8)

    ys, xs = np.where(peaks > 0)
    if len(xs) == 0:
        return [(W//2, H//2)]

    pts = sorted(list(zip(xs, ys)), key=lambda p: interest[p[1], p[0]], reverse=True)
    selected = []
    for (x, y) in pts:
        if all(np.hypot(x - sx, y - sy) >= min_dist for (sx, sy) in selected):
            selected.append((x, y))
    return selected


def extract_regions_of_interest(interest_map, component_maps, n_min=2, n_max=5):
    """Extract ROIs and compute per-region factor scores."""
    H, W = interest_map.shape
    im = _norm01(interest_map)

    spread = float(im.std())
    k_est = 2 + int(np.clip(spread * 8.0, 0, 3))  # 2–5 adaptive
    k = int(np.clip(k_est, n_min, n_max))

    min_dist = int(min(H, W) * 0.22)
    centers = _find_peak_centers(im, min_dist)
    if len(centers) < k:
        k = max(n_min, min(len(centers), n_max))
    centers = centers[:k]

    base_r = int(min(H, W) * 0.14)
    rois = []
    for (cx, cy) in centers:
        r = base_r
        x1, y1 = max(0, cx - r), max(0, cy - r)
        x2, y2 = min(W, cx + r), min(H, cy + r)
        patch = im[y1:y2, x1:x2]
        score = float(patch.mean()) if patch.size else 0.0

        factors = {}
        for key, m in component_maps.items():
            roi_m = m[y1:y2, x1:x2]
            factors[key] = float(roi_m.mean()) if roi_m.size else 0.0

        rois.append(dict(center=(cx, cy),
                         bbox=(x1, y1, x2 - x1, y2 - y1),
                         score=score,
                         factors=factors))

    rois.sort(key=lambda r: r["score"], reverse=True)
    return rois
# ==========================================================
# ROI Overlay Rendering (Shaded Regions + Auto-Text Labels)
# ==========================================================

ROI_COLORS = [
    (255, 80, 80),
    (255, 200, 50),
    (80, 255, 120),
    (80, 200, 255),
    (255, 120, 220),
]

def _measure_text(img, text, font=cv2.FONT_HERSHEY_SIMPLEX, scale=0.6, thickness=1):
    (w, h), _ = cv2.getTextSize(text, font, scale, thickness)
    return w, h

def _draw_label_box(img, x, y, w, h, alpha=0.82):
    """Draw semi-transparent black box for text."""
    H, W = img.shape[:2]
    x2, y2 = min(W, x + w), min(H, y + h)
    roi = img[y:y2, x:x2].astype(np.float32)
    overlay = np.zeros_like(roi)
    blended = (1 - alpha) * roi + alpha * overlay
    img[y:y2, x:x2] = np.clip(blended, 0, 255).astype(np.uint8)
    cv2.rectangle(img, (x, y), (x2, y2), (255, 255, 255), 1, cv2.LINE_AA)

def _put_label_lines(img, x, y, lines, leading=20, color=(255, 255, 255)):
    """Render wrapped text lines inside label box."""
    f = cv2.FONT_HERSHEY_SIMPLEX
    for i, line in enumerate(lines):
        cv2.putText(img, line, (x + 8, y + 18 + i * leading), f, 0.55, color, 1, cv2.LINE_AA)

def _choose_label_rect(img_shape, bbox, used, text_lines, pad=6):
    """Select label placement to avoid overlaps."""
    H, W = img_shape[:2]
    f = cv2.FONT_HERSHEY_SIMPLEX
    line_w = [cv2.getTextSize(t, f, 0.55, 1)[0][0] for t in text_lines]
    box_w = max(140, min(W - 12, max(line_w) + 16))
    box_h = 12 + 20 * len(text_lines)

    x, y, w, h = bbox
    cx, cy = x + w // 2, y + h // 2
    candidates = [
        (x, max(6, y - box_h - 8)),                # above
        (x, min(H - box_h - 6, y + h + 8)),        # below
        (min(W - box_w - 6, x + w + 8), max(6, y)),# right
        (max(6, x - box_w - 8), max(6, y)),        # left
        (min(W - box_w - 6, x + 6), min(H - box_h - 6, y + 6)),  # inside
    ]

    def overlaps(a, b):
        ax1, ay1, ax2, ay2 = a[0], a[1], a[0] + box_w, a[1] + box_h
        bx1, by1, bx2, by2 = b
        return not (ax2 < bx1 or bx2 < ax1 or ay2 < by1 or by2 < ay1)

    for (lx, ly) in candidates:
        lx = int(np.clip(lx, 6, W - box_w - 6))
        ly = int(np.clip(ly, 6, H - box_h - 6))
        rect = (lx, ly, lx + box_w, ly + box_h)
        if not any(overlaps((lx, ly), u) for u in used):
            used.append(rect)
            return lx, ly, box_w, box_h

    lx, ly = 8, min(H - box_h - 8, 8 + len(used) * (box_h + 8))
    rect = (lx, ly, lx + box_w, ly + box_h)
    used.append(rect)
    return lx, ly, box_w, box_h

def _summarize_roi_naturally(i, roi, C, T, S, P, I):
    """Generate short descriptive text for each ROI."""
    f = roi["factors"]
    keys = ["sal", "tex", "con", "line", "prox_focal", "prox_vp"]
    ranked = sorted(keys, key=lambda k: f.get(k, 0.0), reverse=True)
    top = ranked[:2]

    tone_color = (
        "quiet transitions" if C["mean_deltaE"] < 10 else
        "measured contrast" if C["mean_deltaE"] < 20 else
        "confident contrast"
    )

    tex_snip = (
        "a calm surface" if T.get("texture_std", 0.1) < 0.25 else
        "a sense of tactile movement"
    )

    light_snip = (
        f"subtly shaped by light from ~{int(I['light_direction_deg'])}°"
        if I.get("light_direction_deg") is not None else
        "bathed in diffuse light"
    )

    H, W = S["saliency_map"].shape
    x, y, w, h = roi["bbox"]
    cx, cy = x + w / 2, y + h / 2
    vertical = "upper" if cy < H / 2 else "lower"
    horizontal = "left" if cx < W / 2 else "right"
    zone = f"{vertical}-{horizontal}"

    lead = {
        "sal": "Naturally eye-catching,",
        "tex": "Texturally rich,",
        "con": "Colorfully assertive,",
        "line": "Structurally charged,",
        "prox_focal": "Close to the focal center,",
        "prox_vp": "Aligned with perspective,",
    }.get(top[0], "Visually distinct,")

    second = {
        "sal": "it draws attention instantly.",
        "tex": "the surface carries movement.",
        "con": f"tones meet with {tone_color}.",
        "line": "linework guides the gaze.",
        "prox_focal": "it reinforces the main subject.",
        "prox_vp": "the geometry feels ordered.",
    }.get(top[1] if len(top) > 1 else top[0], "the details hold attention.")

    endings = [
        "the area rewards a slower look.",
        f"Here there's {tex_snip}.",
        f"It feels {light_snip}.",
        "it feels resolved yet alive.",
    ]
    tail = endings[i % len(endings)]

    return f"{lead} this {zone} region balances where {second} {tail}"
# ==========================================================
# ROI Overlay Rendering (Shaded Regions + Auto-Text Labels)
# ==========================================================

ROI_COLORS = [
    (255, 80, 80),
    (255, 200, 50),
    (80, 255, 120),
    (80, 200, 255),
    (255, 120, 220),
]

def _measure_text(img, text, font=cv2.FONT_HERSHEY_SIMPLEX, scale=0.6, thickness=1):
    (w, h), _ = cv2.getTextSize(text, font, scale, thickness)
    return w, h

def _draw_label_box(img, x, y, w, h, alpha=0.82):
    """Draw semi-transparent black box for text."""
    H, W = img.shape[:2]
    x2, y2 = min(W, x + w), min(H, y + h)
    roi = img[y:y2, x:x2].astype(np.float32)
    overlay = np.zeros_like(roi)
    blended = (1 - alpha) * roi + alpha * overlay
    img[y:y2, x:x2] = np.clip(blended, 0, 255).astype(np.uint8)
    cv2.rectangle(img, (x, y), (x2, y2), (255, 255, 255), 1, cv2.LINE_AA)

def _put_label_lines(img, x, y, lines, leading=20, color=(255, 255, 255)):
    """Render wrapped text lines inside label box."""
    f = cv2.FONT_HERSHEY_SIMPLEX
    for i, line in enumerate(lines):
        cv2.putText(img, line, (x + 8, y + 18 + i * leading), f, 0.55, color, 1, cv2.LINE_AA)

def _choose_label_rect(img_shape, bbox, used, text_lines, pad=6):
    """Select label placement to avoid overlaps."""
    H, W = img_shape[:2]
    f = cv2.FONT_HERSHEY_SIMPLEX
    line_w = [cv2.getTextSize(t, f, 0.55, 1)[0][0] for t in text_lines]
    box_w = max(140, min(W - 12, max(line_w) + 16))
    box_h = 12 + 20 * len(text_lines)

    x, y, w, h = bbox
    cx, cy = x + w // 2, y + h // 2
    candidates = [
        (x, max(6, y - box_h - 8)),                # above
        (x, min(H - box_h - 6, y + h + 8)),        # below
        (min(W - box_w - 6, x + w + 8), max(6, y)),# right
        (max(6, x - box_w - 8), max(6, y)),        # left
        (min(W - box_w - 6, x + 6), min(H - box_h - 6, y + 6)),  # inside
    ]

    def overlaps(a, b):
        ax1, ay1, ax2, ay2 = a[0], a[1], a[0] + box_w, a[1] + box_h
        bx1, by1, bx2, by2 = b
        return not (ax2 < bx1 or bx2 < ax1 or ay2 < by1 or by2 < ay1)

    for (lx, ly) in candidates:
        lx = int(np.clip(lx, 6, W - box_w - 6))
        ly = int(np.clip(ly, 6, H - box_h - 6))
        rect = (lx, ly, lx + box_w, ly + box_h)
        if not any(overlaps((lx, ly), u) for u in used):
            used.append(rect)
            return lx, ly, box_w, box_h

    lx, ly = 8, min(H - box_h - 8, 8 + len(used) * (box_h + 8))
    rect = (lx, ly, lx + box_w, ly + box_h)
    used.append(rect)
    return lx, ly, box_w, box_h

def _summarize_roi_naturally(i, roi, C, T, S, P, I):
    """Generate short descriptive text for each ROI."""
    f = roi["factors"]
    keys = ["sal", "tex", "con", "line", "prox_focal", "prox_vp"]
    ranked = sorted(keys, key=lambda k: f.get(k, 0.0), reverse=True)
    top = ranked[:2]

    tone_color = (
        "quiet transitions" if C["mean_deltaE"] < 10 else
        "measured contrast" if C["mean_deltaE"] < 20 else
        "confident contrast"
    )

    tex_snip = (
        "a calm surface" if T.get("texture_std", 0.1) < 0.25 else
        "a sense of tactile movement"
    )

    light_snip = (
        f"subtly shaped by light from ~{int(I['light_direction_deg'])}°"
        if I.get("light_direction_deg") is not None else
        "bathed in diffuse light"
    )

    H, W = S["saliency_map"].shape
    x, y, w, h = roi["bbox"]
    cx, cy = x + w / 2, y + h / 2
    vertical = "upper" if cy < H / 2 else "lower"
    horizontal = "left" if cx < W / 2 else "right"
    zone = f"{vertical}-{horizontal}"

    lead = {
        "sal": "Naturally eye-catching,",
        "tex": "Texturally rich,",
        "con": "Colorfully assertive,",
        "line": "Structurally charged,",
        "prox_focal": "Close to the focal center,",
        "prox_vp": "Aligned with perspective,",
    }.get(top[0], "Visually distinct,")

    second = {
        "sal": "it draws attention instantly.",
        "tex": "the surface carries movement.",
        "con": f"tones meet with {tone_color}.",
        "line": "linework guides the gaze.",
        "prox_focal": "it reinforces the main subject.",
        "prox_vp": "the geometry feels ordered.",
    }.get(top[1] if len(top) > 1 else top[0], "the details hold attention.")

    endings = [
        "the area rewards a slower look.",
        f"Here there's {tex_snip}.",
        f"It feels {light_snip}.",
        "it feels resolved yet alive.",
    ]
    tail = endings[i % len(endings)]

    return f"{lead} this {zone} region balances where {second} {tail}"
# ==========================================================
# Drawing: Combined ROI Overlay + Individual ROI Overlays
# ==========================================================

def draw_roi_overlays_with_text(img_rgb, rois, C, T, S, P, I):
    """
    Draw all ROIs with colored shading and auto-placed text summaries.
    Returns:
        combined_img_rgb (uint8 RGB),
        texts (list[str]) per-ROI natural summaries in draw order.
    """
    out = img_rgb.copy().astype(np.float32)
    H, W = out.shape[:2]
    alpha = 0.35
    used_label_rects = []
    texts = []

    for i, roi in enumerate(rois):
        # shaded region
        color = ROI_COLORS[i % len(ROI_COLORS)]
        x, y, w, h = roi["bbox"]
        mask = np.zeros((H, W), np.uint8)
        cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

        tint = np.full_like(out, color, np.uint8)
        out = np.where(mask[:, :, None] > 0,
                       (1 - alpha) * out + alpha * tint.astype(np.float32),
                       out)

        # ROI title + summary
        title = f"ROI {i + 1}"
        summary = _summarize_roi_naturally(i, roi, C, T, S, P, I)
        lines = [title] + wrap_text_lines(summary, width_chars=46)
        texts.append(summary)

        # choose label rect & draw
        lx, ly, bw, bh = _choose_label_rect(out.shape, roi["bbox"], used_label_rects, lines)
        _draw_label_box(out, lx, ly, bw, bh, alpha=0.7)

        # title: accent color with thin black overlap for legibility
        cv2.putText(out, title, (lx + 8, ly + 18),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.65, color, 2, cv2.LINE_AA)
        cv2.putText(out, title, (lx + 8, ly + 18),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 0), 1, cv2.LINE_AA)

        # body lines
        if len(lines) > 1:
            _put_label_lines(out, lx, ly + 6 + 18, lines[1:], leading=20, color=(240, 240, 240))

    return np.clip(out, 0, 255).astype(np.uint8), texts


def draw_single_roi_with_text(img_rgb, roi, idx, C, T, S, P, I):
    """
    Render a single ROI overlay + on-image summary (for per-ROI files).
    Returns:
        single_img_rgb (uint8 RGB),
        summary (str) the natural-language text for this ROI.
    """
    out = img_rgb.copy().astype(np.float32)
    H, W = out.shape[:2]
    alpha = 0.35
    color = ROI_COLORS[idx % len(ROI_COLORS)]
    x, y, w, h = roi["bbox"]

    # shaded region
    mask = np.zeros((H, W), np.uint8)
    cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)
    tint = np.full_like(out, color, np.uint8)
    out = np.where(mask[:, :, None] > 0,
                   (1 - alpha) * out + alpha * tint.astype(np.float32),
                   out)

    # summary
    title = f"ROI {idx + 1}"
    summary = _summarize_roi_naturally(idx, roi, C, T, S, P, I)
    lines = [title] + wrap_text_lines(summary, width_chars=46)

    # pick label location (no previous labels for single)
    used = []
    lx, ly, bw, bh = _choose_label_rect(out.shape, roi["bbox"], used, lines)
    _draw_label_box(out, lx, ly, bw, bh, alpha=0.7)

    cv2.putText(out, title, (lx + 8, ly + 18),
                cv2.FONT_HERSHEY_SIMPLEX, 0.65, color, 2, cv2.LINE_AA)
    cv2.putText(out, title, (lx + 8, ly + 18),
                cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 0), 1, cv2.LINE_AA)
    if len(lines) > 1:
        _put_label_lines(out, lx, ly + 6 + 18, lines[1:], leading=20, color=(240, 240, 240))

    return np.clip(out, 0, 255).astype(np.uint8), summary
# ==========================================================
# Output Generation & Saving (ROIs + Summary + Metrics)
# ==========================================================

def save_roi_outputs(img, rois, C, T, S, P, I, paragraph, out_dir):
    """
    Save:
      • Combined ROI overlay (roi_combined.png)
      • Individual ROI overlays (roi_1.png, roi_2.png, etc.)
      • metrics.json (structured results)
      • curatorial_summary.txt (light + tonal mood summary)
    """
    ensure_dir(out_dir)

    print("💡 Rendering combined ROI overlay…", end=" ")
    combined_img, roi_texts = draw_roi_overlays_with_text(img, rois, C, T, S, P, I)
    cv2.imwrite(os.path.join(out_dir, "roi_combined.png"),
                cv2.cvtColor(combined_img, cv2.COLOR_RGB2BGR))
    print("✓")

    print("💡 Rendering individual ROI overlays…", end=" ")
    for i, roi in enumerate(rois):
        single_img, text = draw_single_roi_with_text(img, roi, i, C, T, S, P, I)
        cv2.imwrite(os.path.join(out_dir, f"roi_{i + 1}.png"),
                    cv2.cvtColor(single_img, cv2.COLOR_RGB2BGR))
    print("✓")

    print("💾 Writing metrics and summaries…", end=" ")
    metrics = dict(
        color=json_safe(C),
        texture=json_safe(T),
        saliency=json_safe(S),
        composition=json_safe(P),
        illumination=json_safe(I),
        rois=json_safe(rois)
    )
    with open(os.path.join(out_dir, "metrics.json"), "w", encoding="utf-8") as f:
        json.dump(metrics, f, indent=2)

    with open(os.path.join(out_dir, "curatorial_summary.txt"), "w", encoding="utf-8") as f:
        f.write(paragraph + "\n")

    print("✓")
    print(f"📁 All ROI outputs saved to: {out_dir}")
# ==========================================================
# Main Function — ROI-Only Analysis + Curatorial Summary
# ==========================================================

def analyze_artwork_v17(path, verbose=True, thorough=True):
    """
    Main entry point:
      • Runs all analyses (color, texture, saliency, composition, light)
      • Dynamically detects 2–5 Regions of Interest (ROIs)
      • Writes per-ROI summaries + combined summary
      • Saves only relevant outputs (ROI overlays + metrics + text)
    """
    start = time.time()
    print(f"🖼️ Loading image: {path}")
    img = load_rgb(path)
    H0, W0 = img.shape[:2]
    print(f"   • Image size: {W0}×{H0}")

    # --- Optional upscaling for fidelity ---
    if thorough:
        img, _ = resize_for_fidelity(img, target_pixels=1_600_000)
        H, W = img.shape[:2]
        if (H, W) != (H0, W0):
            print(f"   • Rescaled for fidelity: {W}×{H}")
    else:
        H, W = H0, W0

    # --- Output directory ---
    out_dir = os.path.splitext(path)[0] + "_analysis"
    ensure_dir(out_dir)

    # --- Analyses ---
    print("🎨 Running color analysis…", end=" "); C = color_analysis(img); print("✓")
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    print("🪶 Running texture analysis…", end=" "); T = texture_analysis(gray); print("✓")
    print("🔥 Running saliency analysis…", end=" "); S = saliency_analysis(img); print("✓")
    print("📐 Running composition analysis…", end=" "); P = composition_analysis(img, S['focal']); print("✓")
    print("💡 Running illumination analysis…", end=" "); I = illumination_analysis(img); print("✓")

    # --- ROI Processing ---
    print("🌐 Building ROI interest map…", end=" ")
    C2, T2, S2, P2, I2 = _prepare_roi_inputs(C, T, S, P, I, shape=img.shape[:2])
    interest = roi_interest_map(C2, T2, S2, P2, I2, shape=img.shape[:2])
    component_maps = _roi_component_maps(C2, T2, S2, P2, I2, shape=img.shape[:2])
    print("✓")

    print("🔍 Extracting regions of interest…", end=" ")
    rois = extract_regions_of_interest(interest, component_maps)
    print(f"{len(rois)} regions ✓")

    # --- Curatorial Summary ---
    print("🖋️ Writing curatorial summary…", end=" ")
    paragraph = light_description(I)
    print("✓")

    # --- Save Outputs ---
    save_roi_outputs(img, rois, C, T, S, P, I, paragraph, out_dir)

    elapsed = time.time() - start
    print(f"\n✅ Complete! Total runtime: {elapsed:.2f}s")
    print(f"📁 Outputs saved to: {out_dir}")
# ==========================================================
# Gamma Correction + Finalization Helpers
# ==========================================================

def safe_gamma_apply_and_save(img_rgb, path):
    """Apply gentle gamma correction before saving (for viewing clarity)."""
    corrected = safe_gamma(img_rgb)
    cv2.imwrite(path, cv2.cvtColor(corrected, cv2.COLOR_RGB2BGR))

def finalize_roi_outputs(img, rois, C, T, S, P, I, paragraph, out_dir):
    """
    Post-process and save all ROI outputs with consistent gamma correction:
      • Combined overlay
      • Individual ROI overlays
      • metrics.json
      • curatorial_summary.txt
    """
    ensure_dir(out_dir)
    print("✨ Finalizing and saving ROI outputs…", end=" ")

    combined_img, _ = draw_roi_overlays_with_text(img, rois, C, T, S, P, I)
    safe_gamma_apply_and_save(combined_img, os.path.join(out_dir, "roi_combined.png"))

    for i, roi in enumerate(rois):
        single_img, _ = draw_single_roi_with_text(img, roi, i, C, T, S, P, I)
        safe_gamma_apply_and_save(single_img, os.path.join(out_dir, f"roi_{i+1}.png"))

    metrics = dict(
        color=json_safe(C),
        texture=json_safe(T),
        saliency=json_safe(S),
        composition=json_safe(P),
        illumination=json_safe(I),
        rois=json_safe(rois)
    )

    with open(os.path.join(out_dir, "metrics.json"), "w", encoding="utf-8") as f:
        json.dump(metrics, f, indent=2)
    with open(os.path.join(out_dir, "curatorial_summary.txt"), "w", encoding="utf-8") as f:
        f.write(paragraph + "\n")

    print("✓")
    print(f"📁 Finalized ROI outputs saved in: {out_dir}")
# ==========================================================
# Entry Point — Run Analyzer
# ==========================================================

if __name__ == "__main__":
    # 🔧 Change this to the artwork you want to analyze:
    IMAGE_PATH = "/Users/alievanayasso/Documents/SlowMA/Raymond-Simboli-.jpg"

    try:
        analyze_artwork_v17(IMAGE_PATH, verbose=True, thorough=True)
    except Exception as e:
        print("\n❌ Error during analysis:")
        import traceback
        traceback.print_exc()
        print("\n💡 Tip: Check that your image path is correct and all dependencies are installed.")



🖼️ Loading image: /Users/alievanayasso/Documents/SlowMA/Raymond-Simboli-.jpg
   • Image size: 1140×797
   • Rescaled for fidelity: 1512×1057
🎨 Running color analysis… ✓
🪶 Running texture analysis… ✓
🔥 Running saliency analysis… ✓
📐 Running composition analysis… ✓
💡 Running illumination analysis… ✓
🌐 Building ROI interest map… ✓
🔍 Extracting regions of interest… 3 regions ✓
🖋️ Writing curatorial summary… ✓
💡 Rendering combined ROI overlay… ✓
💡 Rendering individual ROI overlays… ✓
💾 Writing metrics and summaries… ✓
📁 All ROI outputs saved to: /Users/alievanayasso/Documents/SlowMA/Raymond-Simboli-_analysis

✅ Complete! Total runtime: 3.99s
📁 Outputs saved to: /Users/alievanayasso/Documents/SlowMA/Raymond-Simboli-_analysis
